In [31]:
import pandas as pd
import numpy as np

In [32]:
stock = "sp500"

In [33]:
df_trade=pd.read_pickle(f"{stock}fealab.pkl")
df_trade.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
df_trade['datetime'] = df_trade['datetime'].astype('datetime64[ns]')
df_trade=df_trade[df_trade['datetime']>'2022-12-31']
df_trade.reset_index(inplace=True,drop=True)
df_trade

,instrument,datetime,open,high,low,close,volume,ret_1,label
0,A,2023-01-03,0.021356,0.021371,0.020960,0.021088,0.001547,1.002606,1.010864
1,AAL,2023-01-03,0.001567,0.001559,0.001529,0.001538,0.023920,1.001572,1.066719
2,AAPL,2023-01-03,0.018271,0.018228,0.017487,0.017533,0.122656,0.962595,1.010314
3,ABBV,2023-01-03,0.022791,0.022770,0.022724,0.022845,0.005402,1.004765,1.008067
4,ABT,2023-01-03,0.015457,0.015462,0.015283,0.015327,0.004564,0.998087,1.014875
...,...,...,...,...,...,...,...,...,...
121258,YUM,2023-12-28,0.018350,0.018247,0.018373,0.018309,0.000965,0.997554,1.000000
121259,ZBH,2023-12-28,0.016940,0.016981,0.017020,0.017043,0.000724,1.004128,1.000000
121260,ZBRA,2023-12-28,0.038740,0.038798,0.038729,0.038931,0.000212,0.998405,1.000000
121261,ZION,2023-12-28,0.006000,0.006041,0.006028,0.006083,0.001232,1.003370,1.000000


In [34]:
index_date_unique=df_trade['datetime'].unique()
index_date_unique.shape

(249,)

In [35]:
df=pd.read_csv(f'{stock}/prediction/pred.csv')
df.rename(columns={'dt':'datetime','kdcode':'instrument'},inplace=True)
df['datetime']=df['datetime'].astype('datetime64[ns]')
df_score=pd.DataFrame()
for date in index_date_unique[:-1]:
    loc=np.where(index_date_unique==date)
    nextdate_loc=np.where(index_date_unique==date)[0][0]+1
    for i in range(0,len(df)):
        if df['datetime'][i]== date:
            b={'datetime':index_date_unique[nextdate_loc],'instrument':df['instrument'][i],'score':df['score'][i]}
            df_score=df_score._append(b,ignore_index=True)
x=pd.merge(df_trade,df_score,on=['datetime','instrument'],how='outer')
trade_date_unique=df_trade['datetime'].unique()
df_return=pd.DataFrame()
b={'datetime':trade_date_unique[0],'daily_return':0}
df_return=df_return._append(b,ignore_index=True)
for date in trade_date_unique[1:]:
    x_day=x[x['datetime']==date]
    r_day=x_day.nlargest(9,columns='score').ret_1.mean()
    r_day=r_day-1
    #r_day-=x_day.nsmallest(3,columns='score').close_r.mean()
    b={'datetime':date,'daily_return':r_day}
    df_return=df_return._append(b,ignore_index=True)

In [36]:
portfolio_df_performance = df_return.set_index(['datetime'])
portfolio_df_performance

,daily_return
datetime,
2023-01-03,0.000000
2023-01-04,0.019496
2023-01-05,-0.006820
2023-01-06,0.025546
2023-01-09,0.010218
...,...
2023-12-21,0.005227
2023-12-22,0.004793
2023-12-26,0.000200


In [37]:
ARR = (1 + portfolio_df_performance).cumprod()
ARR

,daily_return
datetime,
2023-01-03,1.000000
2023-01-04,1.019496
2023-01-05,1.012543
2023-01-06,1.038409
2023-01-09,1.049020
...,...
2023-12-21,1.494812
2023-12-22,1.501976
2023-12-26,1.502277


In [38]:
ARR.to_csv(f"ARR/{stock}.csv", index=None)

In [39]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date
## TODO 补充第一年的数据
yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_18636\980728215.py:12: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_18636\980728215.py:20: FutureWarning: 'y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFra

In [40]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,0.51583,0.153801,-0.11158,3.353886,4.622974,2.794464,0.508343,0.009689


THGNN
ASR = ARR / AVol;   CR = ARR / abs(MDD)
        ARR    AVol    MDD    ASR    CR    IR
hs300 -0.015  0.172  -0.152 -0.088 -0.100 -0.003

zz500  0.048  0.128  -0.141  0.375  0.340  0.432

sp500  0.516  0.154  -0.112  3.354  4.623  2.794
   
nas100 0.644  0.204  -0.146  3.147  4.414  2.543
